In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-10 02:34:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-01-10 02:34:35 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

### Create DataFrames to match tables

In [5]:
# Deliverable 2
new_df =df.select(["customer_id","review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
new_df.show()

+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|customer_id|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|   45610553| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
|   14640079| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
|    6111003| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|    1546619|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|   12222213|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|   46018513|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
|   10225065| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|    6356995|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|   352971

In [6]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or 
# greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_count_df = new_df.filter("total_votes>=20")
votes_count_df.show()

+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|customer_id|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|   15365576|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|   28770559|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
|   35225823| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|   24740219|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|   26124872|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|   16452718|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|   40567300|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|   52695075|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
|   115872

In [7]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
new_filter_df = votes_count_df.filter("helpful_votes/total_votes>=0.50")
new_filter_df.show()

+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|customer_id|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|   15365576|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|   28770559|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
|   35225823| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|   24740219|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|   26124872|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|   16452718|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|   40567300|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|   49287335|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|   509841

In [8]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_filter_df = new_filter_df.filter(new_filter_df["vine"]=="Y")
vine_filter_df.show()

+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|customer_id|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|   42689422|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|   29182364|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|   19467797|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|   52081784|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|   17421494|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
|   50273418| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|   50027179|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
|   50258811| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|   283226

In [9]:
#Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
non_vine_filter_df = new_filter_df.filter(new_filter_df["vine"]=="N")
non_vine_filter_df.show()

+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|customer_id|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+-----------+----+-----------------+
|   15365576|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|   28770559|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
|   35225823| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|   24740219|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|   26124872|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|   16452718|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|   40567300|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|   49287335|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|   509841

In [10]:
# Determine the total number of reviews, 
total_reviews_count = new_df.count()
print(total_reviews_count)



904765


In [12]:
#total number of reviews for over 50%
vine_reviews_count = vine_filter_df.count()
print(vine_reviews_count)

non_vine_reviews_count = non_vine_filter_df.count()
print(non_vine_reviews_count)

60
14477


In [17]:
#the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
star_reviews_paid_df = vine_filter_df.filter(vine_filter_df["star_rating"]=="5")
star_reviews_paid_count = star_reviews_paid_df.count()
print(star_reviews_paid_count)

star_reviews_nonpaid_df = non_vine_filter_df.filter(non_vine_filter_df["star_rating"]=="5")
star_reviews_nonpaid_count = star_reviews_nonpaid_df.count()
print(star_reviews_nonpaid_count)

34
8212


In [19]:
vine_reviews_perc = star_reviews_paid_count/vine_reviews_count * 100
print(vine_reviews_perc)

non_vine_reviews_perc = star_reviews_nonpaid_count/non_vine_reviews_count * 100
print(non_vine_reviews_perc)

56.666666666666664
56.72445948746287


In [42]:
#additional analysis
new_count = new_df.count()
print(new_count)

904765


In [43]:
new_vine_df = new_df.filter(new_df["vine"]=="Y")
new_vine_count = new_vine_df.filter(new_vine_df["helpful_votes"]>="50").count()
print(new_vine_count)

new_vine_perc = new_vine_count/new_count * 100
print(new_vine_perc)

14
0.0015473631274419324


In [44]:
new_nonvine_df = new_df.filter(new_df["vine"]=="N")
new_nonvine_count = new_nonvine_df.filter(new_nonvine_df["helpful_votes"]>="50").count()
print(new_nonvine_count)

new_nonvine_perc = new_nonvine_count/new_count * 100
print(new_nonvine_perc)

3153
0.34848828148745803
